# Manila - Jeepney Route Connection and Genetic Algorithm

## Route Connection and Genetic Algorithm Functions

### Setting Up

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import networkx as nx
import shapely
import folium
import geojson
import math
import osmnx as ox
from rtree import index as rtree_index
import pickle
import copy

from shapely.ops import unary_union
from shapely.geometry import Polygon, MultiPolygon, LineString, Point
from geopy.distance import geodesic

from __future__ import absolute_import, division
from math import radians, sin, cos, sqrt, atan2, exp, log
import webbrowser
import random
from scipy.spatial import KDTree
from scipy.spatial.distance import euclidean

ox.settings.log_console=True
ox.settings.use_cache=True

WALKING_DISTANCES = [300,550,800]
MAX_DISTANCE = 15
CONNECTION_TYPES = ["Default", "Area", "Degree", "Mixed"]

In [ ]:
# Defining classes for the dataframes      
class stopCandidate:
    def __init__(self, lat, long, isTranspo, id, area):
        self.lat = lat
        self.long = long
        self.isTranspo = isTranspo
        self.enabled = False
        self.id = id #node ID
        self.area = area
        self.degree = 0
        
    def enable(self):
        self.enabled = True
        
    def disable(self):
        self.enabled = False
        
    def getLat(self):
        return self.lat
    
    def getLong(self):
        return self.long
    
    def getArea(self):
        return self.area
    
    def getDegree(self):
        return self.degree
    
class networkObj():
    def __init__(self, routes, stops, graph, conn_type, walk_distance):
        self.routes = routes
        self.stops = stops
        self.fitness_score = 0
        self.graph = graph
        self.conn_type = conn_type
        self.walk_distance = walk_distance

In [ ]:
# For units
def degrees_to_meters(angle_degrees):
    return angle_degrees * 6371000 * math.pi / 180

def meters_to_degrees(distance_meters):
    return distance_meters / 6371000 * 180 / math.pi

In [ ]:
# Import and Export networks or graphs to pickle
def export_networks(networks, path):
    with open(path, 'wb') as f:
        pickle.dump(networks, f)

def import_networks(path):
    with open(path, 'rb') as f:
        routes = pickle.load(f)
    return routes

#### Graph and Features

In [ ]:
# NOTE: SELECT THE CITY HERE, COMMENT OUT THE REMAINING CITIES
select_city = "Manila, Philippines"
city_file = 'map/Manila.graphml'


# GENERATION OF MAIN CITY GRAPH
# IF FIRST TIME RUNNING, RUN THIS CODE TO GENERATE THE GRAPH
def generate_graph():
    mode = 'drive'
    graph = ox.graph_from_place(select_city, network_type = mode) # Generate graph of Metro manila
    ox.save_graphml(graph, city_file) # Save it as a file

def load_graph():
    graph = ox.load_graphml(city_file)
    
    print("Graph loaded successfully")
    print("NUMBER OF EDGES: ", graph.number_of_edges())
    print("NUMBER OF NODES: ", graph.number_of_nodes())
    print('\n')
    return graph

# NOTE: Only run this if you do not have the graph
generate_graph()

# THIS IS THE MAIN GRAPH FOR THE CITY TO BE USED FOR ALL FUNCTIONS
CITY_GRAPH = load_graph()

In [ ]:
### For Filtering the roads and other features
# GETTING ROADS AND WATERWAYS

# Get all the roads in Manila
road = ox.graph_to_gdfs(CITY_GRAPH,nodes=False, edges=True)


# Get all the roads that are not junctions (ex. Roundabouts, intersection, etc.)
filtered_roads = road[road['junction'].isna()]

# Separate roads whose widths are only one value and those that are more than 1 (lists)
rows_with_lists = filtered_roads[filtered_roads['highway'].apply(lambda x: isinstance(x, list))]
rows_with_strings = filtered_roads[filtered_roads['highway'].apply(lambda x: isinstance(x, str))]

filter_options = ['primary', 'secondary', 'tertiary', 'trunk', 'unclassified']
separation_options = ['primary', 'secondary', 'tertiary', 'unclassified']

# Get the roads whose widths are above the threshold
def check_list(lst):
    return any(x in filter_options for x in lst)

# Download OpenStreetMap data for the area of interest
waterways = ox.features_from_place(select_city, tags={'waterway': True})
filtered_rivers = waterways[waterways['waterway'].isin(['river'])]
filtered_streams = waterways[waterways['waterway'].isin(['stream'])]

# Get all the roads with the allowed road types
filtered_roads_strings = rows_with_strings.loc[rows_with_strings['highway'].isin(filter_options)] 
filtered_roads_lists = rows_with_lists[rows_with_lists['highway'].apply(check_list)]

In [ ]:
# This function will find which road or river intersects between amenities
# Create spatial index
filtered_roads_strings_sindex = filtered_roads_strings.sindex
filtered_roads_lists_sindex = filtered_roads_lists.sindex
filtered_rivers_sindex = filtered_rivers.sindex
filtered_streams_sindex = filtered_streams.sindex

def find_intersecting_features(line):
    # Check intersection with filtered roads
    possible_matches_roads = filtered_roads_strings.iloc[list(filtered_roads_strings_sindex.intersection(line.bounds))]
    for index, row in possible_matches_roads.iterrows():
        if line.intersects(row['geometry']) and row['highway'] in separation_options:
            return True

    possible_matches_lists = filtered_roads_lists.iloc[list(filtered_roads_lists_sindex.intersection(line.bounds))]
    for index, row in possible_matches_lists.iterrows():
        if line.intersects(row['geometry']):
            list_highway = row['highway']
            if any(x in separation_options for x in list_highway):
                return True
    
    # Check intersection with filtered rivers
    possible_matches_rivers = filtered_rivers.iloc[list(filtered_rivers_sindex.intersection(line.bounds))]
    for index, row in possible_matches_rivers.iterrows():
        if line.intersects(row['geometry']):
            return True

    # Check intersection with filtered streams
    possible_matches_streams = filtered_streams.iloc[list(filtered_streams_sindex.intersection(line.bounds))]
    for index, row in possible_matches_streams.iterrows():
        if line.intersects(row['geometry']):
            return True
    
    return False

### Stop Placement

In [ ]:
# ADD POINTS TO NX GRAPH
# Function to add only points to the networkX graph
# The other functions focuses on adding polygons, this function just iterates and adds points

def add_points_to_graph(graph, graph_to_add):
    for node_key, node_data in graph.nodes.items():
        if 'geometry' in node_data and node_data['geometry'].geom_type in ['Point']:   
            graph_to_add.add_node(node_key, geometry=node_data['geometry'], name=node_data['name'], lat=node_data['lat'], amenity=node_data['amenity'],
                                lon=node_data['lon'])
                
                

In [ ]:

# CREATING STOPS
# It should return a list of coordinates/nodes for stop and a graph of stops
# if residential area, check if the population density

# Global Variables used:
# list_of_stops - List of stops
# graph_of_stops - graph of all stops placed
# CITY_GRAPH - graph of the city road networks
import random

def place_stops_on_roads(amenity_graph, graph_of_stops, list_of_stops):
    global CITY_GRAPH  
    for node_key, node_data in amenity_graph.nodes(data=True):
        # All tranportation points are automatically stops
        if node_data['geometry'].geom_type in ['Point']:
            if node_data['amenity'] == 'transportation':
                nearest_node = ox.distance.nearest_nodes(CITY_GRAPH, node_data['lon'], node_data['lat'])
                
                # If there is an existing node in the main graph, then add it to the list and the stop graph
                if nearest_node is not None:
                    if not graph_of_stops.has_node(nearest_node):
                        lon = CITY_GRAPH.nodes[nearest_node]['x']
                        lat = CITY_GRAPH.nodes[nearest_node]['y']
                        isTranspo = True
                        graph_of_stops.add_node(nearest_node, lon=lon, lat=lat, isTranspo=isTranspo)
                        list_of_stops.append(stopCandidate(CITY_GRAPH.nodes[nearest_node]['y'], CITY_GRAPH.nodes[nearest_node]['x'], True, nearest_node, 0))
        
        else:
            # Calculate the number of stops based on node size and population density
            num_stops, node_size = calculate_num_stops(node_key, node_data)
            
            buffer_poly = node_data['geometry'].buffer(meters_to_degrees(30))
            # Get the roads surrounding and inside the node polygons
            relevant_edges = get_relevant_edges(buffer_poly)
            
            # Place stops randomly on these roads
            place_stops_along_edges(relevant_edges, buffer_poly, num_stops, node_size, graph_of_stops, list_of_stops)

def calculate_num_stops(node_key, node_data):
    # Example calculation based on node size and population density
    node_size = degrees_to_meters(node_data['geometry'].area) # Size of the node polygon
    # Adjust factors and formula as needed
    num_stops = 0
    
    if node_key in pop_graph:
        pop_density = pop_graph.nodes[node_key]['density']
        num_stops = node_size * pop_density / 10000000  # Adjust this factor as needed
        
        if num_stops < 1:
            num_stops = 1
        elif num_stops > 3:
            num_stops = 3
    else:
        list_sum = len(node_data['amenity_points'])

        if list_sum > 0:
            num_stops = 5
        else:
            num_stops = 3
        
    return int(num_stops), node_size


# Create spatial index
filtered_roads_strings_sindex = filtered_roads_strings.sindex
filtered_roads_lists_sindex = filtered_roads_lists.sindex
def get_relevant_edges(polygon):
    relevant_edges = []
    
    # Check intersection with filtered roads
    possible_matches_roads = filtered_roads_strings.iloc[list(filtered_roads_strings_sindex.intersection(polygon.bounds))]
    for index, row in possible_matches_roads.iterrows():
        if polygon.intersects(row['geometry']) and row['highway'] in ['primary', 'secondary', 'tertiary', 'residential']:
            relevant_edges.append(row)

    possible_matches_lists = filtered_roads_lists.iloc[list(filtered_roads_lists_sindex.intersection(polygon.bounds))]
    for index, row in possible_matches_lists.iterrows():
        if polygon.intersects(row['geometry']):
            list_highway = row['highway']
            if any(x in ['primary', 'secondary', 'tertiary', 'residential'] for x in list_highway):
                relevant_edges.append(row)
    return relevant_edges

def place_stops_along_edges(edges, polygon, num_stops, node_size, graph_of_stops, list_of_stops):
    # Place stops randomly along the edges within the polygon
    
    if len(edges) > 0:
        for _ in range(num_stops):
            edge = random.choice(edges)
            # Calculate the intersection between the edge and the polygon
            intersecting_line = edge['geometry'].intersection(polygon)
            if intersecting_line.is_empty:
                continue

            # Calculate the length of the intersecting part of the edge
            intersecting_length = intersecting_line.length

            # Generate a random position along the intersecting part of the edge
            random_position = random.uniform(0, intersecting_length)

            # Calculate the coordinate along the edge at the random position
            stop_location = calculate_coordinate_along_edge(intersecting_line, random_position)
            #print("Stop placed at:", stop_location)
            
            nearest_node = ox.distance.nearest_nodes(CITY_GRAPH, stop_location[0], stop_location[1])
            
            # If there is an existing node in the main graph, then add it to the list and the stop graph
            if nearest_node is not None:
                if not graph_of_stops.has_node(nearest_node):
                    lon = CITY_GRAPH.nodes[nearest_node]['x']
                    lat = CITY_GRAPH.nodes[nearest_node]['y']
                    isTranspo = False
                    graph_of_stops.add_node(nearest_node, lon=lon, lat=lat, isTranspo=isTranspo)
                    list_of_stops.append(stopCandidate(CITY_GRAPH.nodes[nearest_node]['y'], CITY_GRAPH.nodes[nearest_node]['x'], False,  nearest_node, node_size))
            else:
                _ -= 1
        
            

def calculate_coordinate_along_edge(edge, position):
    # Calculate the coordinate along the edge at the given position
    point = edge.interpolate(position)
    return point.x, point.y
    

### Route Network Generation

In [ ]:
# V2 - Graph with the snapping function
# Generate Route Network from connected routes

# Global Variables used:
# graph_of_stops - Graph of stops that will be used to create routes
def generate_route_network(stop_nodes, max_walking_dist, max_stops, max_routes, graph_of_stops, city_area_sum, connection_type="Default"):
    global route_count
    overall_graph = nx.Graph()
    
    stop_node_coordinates = [[n.lat, n.long] for n in stop_nodes]
    stop_nodes_kd_tree = KDTree(stop_node_coordinates)
    next_nodes = [n for n in stop_nodes]
    enable_stop_nodes(next_nodes)
    route_network = []
    num_routes = 0 # Count number of routes

    while num_routes < max_routes:
        next_nodes = [n for n in stop_nodes] # Resets the list of nodes so that nodes can be reused in a different
        selected_node = random.choice(next_nodes) # For the first node
        next_nodes.remove(selected_node)
        route_gen = generate_route(selected_node, next_nodes, stop_nodes_kd_tree, max_walking_dist, connection_type, max_stops, overall_graph, city_area_sum)
        
        if len(route_gen) > 1:
            # A route is a list of connections between two nodes
            snap_route_to_road(route_gen, overall_graph, graph_of_stops)
            route_count += 1
            
            #snapped_edges = list(snapped_route.edges(data='road_path', default=1))
            #snapped_route = connect_snapped_edges(snapped_edges)
            route_network.append(route_gen)   
            num_routes += 1
               
    return route_network, overall_graph

def snap_route_to_road(route, overall_graph, graph_of_stops):
    global connection_count
    
    # Directly add nodes based on node identifiers
    for connection in route:
        overall_graph.add_node(connection[0], **graph_of_stops.nodes[connection[0]]) # The origin
        overall_graph.add_node(connection[-1], **graph_of_stops.nodes[connection[-1]]) # The destination
        
        name = f"{connection[0]}_{connection[-1]}" # "node1_node2" as name
        
        distance_travelled = 0
        # Get the total distance from point A to point B
        for i in range(len(connection)-1):
            node_data = CITY_GRAPH.nodes[connection[i]]
            next_node_data = CITY_GRAPH.nodes[connection[i+1]]
            distance_travelled += haversine(node_data['y'], node_data['x'], next_node_data['y'], next_node_data['x'])
        
        overall_graph.add_edge(connection[0], connection[-1], road_path=connection, edge_name=name, edge_id = connection_count, route_id = route_count, distance = distance_travelled) # Add edge
        connection_count += 1 # Global variable - increment the number of connections
        
        
        if not overall_graph.has_node(connection[0]):
                print("missing ", connection[0], " in route")
        
        if not overall_graph.has_node(connection[-1]):
                print("missing ", connection[-1], " in route ")


# Generate route from stop nodes
def generate_route(source, next_nodes, stop_nodes_kd_tree, max_walking_dist, connection_type, max_stops, network_graph, city_area_sum):
    short_route_list = [] # List of nx.shortest_path results
    totalDistance = 0
    orig_node = source
    num_stops = 0 # Count number of stops
    
    # CONFIGURATION
    max_tries = 3 # This is the max number of tries before breaking the loop || To avoid longer runtimes
    current_tries = 0

    while totalDistance < MAX_DISTANCE and num_stops < max_stops:
        
        #print(f"Selected node is {selected_node.getLat()}, {selected_node.getLong()}")
        enable_surrounding_nodes(next_nodes)
        disable_surrounding_nodes(next_nodes, orig_node, max_walking_dist)
        enabled_nodes = [n for n in next_nodes if n.enabled]
        if len(enabled_nodes) == 0:
            break
        
        #print(f"{len(enabled_nodes)} nodes out of {len(next_nodes)}")
        dest_node = get_enabled_node_with_highest_edge_probability(orig_node, enabled_nodes, connection_type, city_area_sum) # Getting the destination node
        
        if (dest_node == None or dest_node.id == orig_node.id):
            break
        
        next_nodes.remove(dest_node) # Remove it as a candidate
        
        connection_edge = network_graph.has_edge(orig_node.id, dest_node.id) # This is to check if there is already an existing edge. If true, then it should not connect
        
        if not nx.has_path(CITY_GRAPH, orig_node.id, dest_node.id) or connection_edge:
            current_tries += 1
            if current_tries == max_tries:
                break
        else:
            shortest_route = nx.shortest_path(CITY_GRAPH, orig_node.id, dest_node.id)
            distance_travelled = 0
            # Get the total distance from point A to point B
            for i in range(len(shortest_route)-1):
                node_data = CITY_GRAPH.nodes[shortest_route[i]]
                next_node_data = CITY_GRAPH.nodes[shortest_route[i+1]]
                
                distance_travelled += haversine(node_data['y'], node_data['x'], next_node_data['y'], next_node_data['x'])

            # Checks if it does not exceed the max distance
            if totalDistance + distance_travelled <= MAX_DISTANCE:
                
                # Updating local degree count used for connection probability
                orig_node.degree += 1
                dest_node.degree += 1
                
                totalDistance += distance_travelled
                short_route_list.append(shortest_route)
                num_stops += 1
                
                orig_node = dest_node # Now change the origin to the destination
            else:
                break
    if len(short_route_list) > 4 and totalDistance > 7:
        print(f"# OF CONNECTIONS AND TOTAL DISTANCE: {len(short_route_list)} - {totalDistance}")
        return short_route_list
    
    else:
        return []

# Disable surrounding nodes
def disable_surrounding_nodes(next_nodes, source_node, max_distance):
    max_radius = 2000 # This is max radius in which all nodes outside will be disabled
    source = (source_node.getLat(), source_node.getLong())
    
    for node in next_nodes:
        point = (node.getLat(), node.getLong())
        distance_to_source = geodesic(source, point).meters
        if distance_to_source <= max_distance or distance_to_source > max_radius:
            node.disable()
            
# Enable surrounding nodes
def enable_surrounding_nodes(next_nodes):
    for node in next_nodes:
        node.enable()
        
def get_enabled_node_with_highest_edge_probability(source_node, enabled_nodes, connection_type, city_area_sum):
    highest_edge_prob = 0
    highest_edge_prob_node = None

    prob_list = []
    for n in enabled_nodes:
        # TODO: fix probability
        edge_prob = get_edge_probability(source_node, n, len(enabled_nodes), connection_type, city_area_sum)
        prob_list.append(edge_prob)
    
    
    min_score = min(prob_list)
    if min_score < 0: # Shift the scores to ensure all are positive
        prob_list = [score - min_score for score in prob_list]
    total = sum(prob_list)
    selection_p = [score / total for score in prob_list]
    
    chosen_node = np.random.choice(enabled_nodes, 1, p=selection_p)[0]    

    return chosen_node

# Probabilities of candidate nodes based on distance, area, node degree, and if transpo stop
def get_edge_probability(source, destination, normalization_factor, connection_type, city_area_sum):
    source_coord = [source.getLat(), source.getLong()]
    dest_coord = [destination.getLat(), destination.getLong()]

    base_prob = exp(-(euclidean(source_coord, dest_coord))) / float(normalization_factor)

    if connection_type == "Default":
        if destination.isTranspo:
            return base_prob * 1.5
        return base_prob
    elif connection_type == "Area":
        if destination.isTranspo:
            return base_prob * 1.5
        return base_prob * (1 + (destination.getArea() / city_area_sum))
    elif connection_type == "Degree":
        if destination.isTranspo:
            return base_prob * 1.5 * (1 + (destination.getDegree() / 10))
        return base_prob * (1 + (destination.getDegree() / 10))

def radius(stops):
    circles = []
    for stop in stops:
        stop_point = Point(stop[1], stop[0])  # Create a Point object from [lat, lon] coordinates
        circle = stop_point.buffer(radius / 111000)  # Buffer the Point to create a circle (assuming 1 degree is approximately 111000 meters)
        circles.append(circle)
    return circles

def enable_stop_nodes(stop_nodes):
    for n in stop_nodes:
        n.enable()

def all_nodes_disabled(stop_nodes):
    return get_num_disabled(stop_nodes) == len(stop_nodes)

def get_num_disabled(stop_nodes):
    return sum(1 for n in stop_nodes if not n.enabled)

def haversine(lat1, lon1, lat2, lon2):
    # Use geopy's geodesic function to calculate the distance
    distance = geodesic((lat1, lon1), (lat2, lon2)).kilometers
    return distance

# Markers for visualization purposes
def add_markers(used_stops, network_graph):
    for stop in used_stops:
        #popup_text = f"Name: {stop.name}<br>Type: {stop.a_type}<br>Coordinates: {stop.getLat()}, {stop.getLong()}"
        lat = network_graph.nodes[stop]['lat']
        long = network_graph.nodes[stop]['lon']
        folium.Marker(location=[lat, long]).add_to(m)
        
def add_stops_to_list(routes):
    used_stops = []
    for route in routes:
        for conn in route:
            if conn[0] not in used_stops:
                used_stops.append(conn[0])
            if conn[-1] not in used_stops:
                used_stops.append(conn[-1])
    return used_stops

In [ ]:
#TODO: CONNECT THE ROUTE REVERSAL
# Reverse route traversal
def get_reverse_route(network):
    reverse_route_network = []
    
    for route in network.routes:
        index = len(route)-1
        
        reverse_route = []
        totalDistance = 0
        while index >= 0:
            connection = route[index] # Get the connection
            rev_origin = connection[-1]
            rev_dest = connection[0]
            
            if nx.has_path(CITY_GRAPH, rev_origin, rev_dest):
                rev_path = nx.shortest_path(CITY_GRAPH, rev_origin, rev_dest) # Get the path
                
                distance_travelled = 0
                # Get the total distance from point A to point B
                for i in range(len(rev_path)-1):
                    node_data = CITY_GRAPH.nodes[rev_path[i]]
                    next_node_data = CITY_GRAPH.nodes[rev_path[i+1]]
                    distance_travelled += haversine(node_data['y'], node_data['x'], next_node_data['y'], next_node_data['x'])
                    
                totalDistance += distance_travelled
                
                reverse_route.append(rev_path)
                index -= 1
            else:
                # there is no reverse route for this so append nothing to the 
                reverse_route_network.append([])
                break
    
        # Checks if it does not exceed the max distance
        if totalDistance <= MAX_DISTANCE:
            reverse_route_network.append(reverse_route)
        else:
            reverse_route_network.append([])
            
    return reverse_route_network

### Genetic Algorithm

In [ ]:
# GA VERSION 2 - Finding the optimal Network
# TODO: WORKING IN PROGRESS - Find a way to make sure that it will stop when the scores start to plateau, and make sure that crossover children will not be the same

# This implementation takes only 2 parents from the whole generation and generates the population from them
# Instead of the what's in the paper that says the whole population will go through crossovers and mutations
# Cite Nayeem et al for GA with elitism and growing population size
def perform_genetic_algorithm(network_population, graph_of_stops, population_size, num_elites, num_generations, mutation_probability, 
                              num_mutations_probabilities, num_crossovers_probabilities, mutation_threshold_dist, num_failure_removal,
                                      weight_random_failure, weight_targeted_failure, weight_connectivity, optimal_fitness_score,
                              with_elitism=False, with_growing_population=False, num_mutations_per_generation=1):
    
    max_fitness_score = 0 # This is the max score of the current population
    max_score_list = [] # This is to store all the max scores of each generation
    
    generation_num = 1
    # This will continue to loop until the generation with the max score is equal to the optimal score
    #while True:
    for _ in range(5000): # Test only
        print(f"Generation {generation_num}", flush=True)
        
        # Evaluate the fitness of each network in the population
        for network in network_population:
            network_graph = network.graph
            if network.fitness_score == 0:
                network.fitness_score = compute_fitness_score(network_graph, num_failure_removal, weight_random_failure, weight_targeted_failure, weight_connectivity)
            
        # Sort the network population by fitness score
        sorted_network_population = sorted(network_population, key=lambda x: x.fitness_score, reverse=True)
        fitness_scores = [network.fitness_score for network in sorted_network_population]
        
        # for printing purposes in order
        for network in sorted_network_population:
            print(f"Network Score {network.fitness_score}", flush=True)
        
        # The max network score of this generation
        max_fitness_score = max(fitness_scores)
        max_score_list.append(max_fitness_score)
        print(f"Generation {generation_num} Max Score: {max_fitness_score}", flush=True)
        
        # The average network score of this generation
        total_score = sum(fitness_scores)
        average_score = total_score / len(fitness_scores)
        print(f"Generation {generation_num} Average Score: {average_score}", flush=True)
        
        # Check if the score is optimal. If it is, then stop
        if max_fitness_score >= optimal_fitness_score:
            print(f"WE FOUND THE OPTIMAL NETWORK IN GENERATION {generation_num}")
            most_optimal_network = sorted_network_population[0]
            print(f"Fitness Score of the most optimal network: {most_optimal_network.fitness_score}")
            break
        
        
        # Choosing 10% of the networks to be parents using Roulette Wheel Selection
        print("Choosing parents...", flush=True)
        
        #Getting the number of parents to be selected
        num_parents = int(len(network_population) * 0.10)
        if num_parents % 2 == 1:
            num_parents += 1
        
        # List of parents
        parent_networks = []
        print("NUM PARENTS ", num_parents)
        for i in range(num_parents):
            # Get the list of all fitness scores
            fitness_scores = [network.fitness_score for network in sorted_network_population]
            
            # Shift the scores to ensure all are positive
            min_score = min(fitness_scores)
            if min_score < 0:
                fitness_scores = [score - min_score for score in fitness_scores]
            
            # Get the probabilities
            total = sum(fitness_scores)
            selection_p = [score / total for score in fitness_scores]
            
            # Getting the parent
            chosen_parent = np.random.choice(sorted_network_population, 1, p=selection_p)[0]
            print(f"parent {i} Score, ", chosen_parent.fitness_score, flush=True)
            sorted_network_population.remove(chosen_parent)
            parent_networks.append(chosen_parent)
            
            
        # Add back all the removed parent networks
        sorted_network_population.extend(parent_networks)
        
        # Sort the networks by fitness function again
        sorted_network_population = sorted(network_population, key=lambda x: x.fitness_score, reverse=True)
        
        # Pairing the parents
        parent_pairs = [parent_networks[i:i+2] for i in range(0, len(parent_networks), 2)]

        # Each parent pair will now produce children
        print("GETTING CHILDREN", flush=True)
        children_networks = []
        for pair in parent_pairs:
            parent1 = pair[0]
            parent2 = pair[1]
                   
            # Get 2 children from crossovers between the two parents
            child1, child2 = crossover_split_index(parent1, parent2)
            
            # Getting the number of mutations
            index_array = list(range(len(num_mutations_probabilities)))
            num_mutations = np.random.choice(index_array, 1, p=num_mutations_probabilities)[0]

            for j in range(num_mutations):
                # Apply mutations to the children based on mutation probability hyperparameter
                if np.random.rand() < mutation_probability:
                    mutate(child1, graph_of_stops)
                    
                if np.random.rand() < mutation_probability:
                    mutate(child2, graph_of_stops)
            
            # Add the children to the list of children
            children_networks.append(child1)
            children_networks.append(child2)
            
        
        # Preparing the next generation
        
        # Remove the lowest scored networks and replace it with the children
        network_population = sorted_network_population[:-len(children_networks)]
        network_population.extend(children_networks)
        
        # Increment the generation number
        generation_num += 1
        print()

    print(f"Highest score of all generations: {max(max_score_list)}")
    return network_population

In [ ]:
# V2 - Crossover routes should not have the same routes
# CROSSOVER SPLIT INDEX FUNCTION

def crossover_split_index(network1, network2):
    # Split both networks based on which routes do not have the same connections
    # network.routes is a list of routes or list of lists of shortest_path
    routes_same1 = []
    routes_not_same1 = []
    
    for route in network1.routes:
        not_same = True
        
        for connection in route:
            if network2.graph.has_edge(connection[0], connection[-1]):
                not_same = False
                continue
            
        if not_same:
            routes_not_same1.append(route)
        else:
            routes_same1.append(route)
    
    routes_same2 = []
    routes_not_same2 = []
    for route in network2.routes:
        not_same = True
        
        for connection in route:
            if network1.graph.has_edge(connection[0], connection[-1]):
                not_same = False
                continue
            
        if not_same:
            routes_not_same2.append(route)
        else:
            routes_same2.append(route)
    
    # Create new graphs for the left and right sides
    route_graph1 = nx.Graph()
    route_graph2 = nx.Graph()
    
    route_network1 = []
    route_network2 = []
    
    used_stops1 = []
    used_stops2 = []
    
    conn_type1 = network1.conn_type
    conn_type2 = network2.conn_type

    walk_type1 = network1.walk_distance
    walk_type2 = network2.walk_distance
    
    # If all have similar connections, then do not split
    if len(routes_not_same1) == 0:
        test_graph_net1 = network1.graph.copy()
        test_routes_net1 = [copy.deepcopy(r) for r in network1.routes]
        test_stops_net1 = [copy.deepcopy(s) for s in network1.stops]
        child1 = networkObj(test_routes_net1, test_stops_net1, test_graph_net1, network1.conn_type, network1.walk_distance)

        test_graph_net2 = network2.graph.copy()
        test_routes_net2 = [copy.deepcopy(r) for r in network2.routes]
        test_stops_net2 = [copy.deepcopy(s) for s in network2.stops]
        child2 = networkObj(test_routes_net2, test_stops_net2, test_graph_net2, network2.conn_type, network2.walk_distance)
        
        print("THERE ARE NO DIFFERENT ROUTES")
        
    # If all routes of both networks are different, then split at a random index
    elif len(routes_same1) == 0:
        # Split both networks at a random index
        # network.routes is a list of routes or list of lists of shortest_path
        if len(network1.routes) < len(network2.routes):
            split_index = random.randint(0, len(network2.routes)-1)
        else:
            split_index = random.randint(0, len(network1.routes)-1)
            
        count = 0
        for route in network1.routes:
            if count < split_index: # if 0-split_index -> child1 graph
                for connection in route:
                    route_graph1.add_node(connection[0], **network1.graph.nodes[connection[0]])
                    if connection[0] not in used_stops1:
                        used_stops1.append(connection[0])
                        
                    route_graph1.add_node(connection[-1], **network1.graph.nodes[connection[-1]])
                    if connection[-1] not in used_stops1:
                        used_stops1.append(connection[-1])
                    
                    route_graph1.add_edge(connection[0], connection[-1], **network1.graph.get_edge_data(connection[0], connection[-1]))
                route_network1.append(route.copy())
                
                    
            else: # else its for child2 graph
                for connection in route:
                    route_graph2.add_node(connection[0], **network1.graph.nodes[connection[0]])
                    if connection[0] not in used_stops2:
                        used_stops2.append(connection[0])
                        
                    route_graph2.add_node(connection[-1], **network1.graph.nodes[connection[-1]])
                    if connection[-1] not in used_stops2:
                        used_stops2.append(connection[-1])
                    
                    route_graph2.add_edge(connection[0], connection[-1], **network1.graph.get_edge_data(connection[0], connection[-1]))
                route_network2.append(route.copy())
            count += 1
            
        count = 0
        for route in network2.routes:
            if count >= split_index: # if split_index-end -> child1 graph
                for connection in route:
                    route_graph1.add_node(connection[0], **network2.graph.nodes[connection[0]])
                    if connection[0] not in used_stops1:
                        used_stops1.append(connection[0])
                        
                    route_graph1.add_node(connection[-1], **network2.graph.nodes[connection[-1]])
                    if connection[-1] not in used_stops1:
                        used_stops1.append(connection[-1])
                    
                    route_graph1.add_edge(connection[0], connection[-1], **network2.graph.get_edge_data(connection[0], connection[-1]))
                route_network1.append(route.copy())
                    
                    
            else: # else its for child2 graph
                for connection in route:
                    route_graph2.add_node(connection[0], **network2.graph.nodes[connection[0]])
                    if connection[0] not in used_stops2:
                        used_stops2.append(connection[0])
                        
                    route_graph2.add_node(connection[-1], **network2.graph.nodes[connection[-1]])
                    if connection[-1] not in used_stops2:
                        used_stops2.append(connection[-1])
                    
                    route_graph2.add_edge(connection[0], connection[-1], **network2.graph.get_edge_data(connection[0], connection[-1]))
                route_network2.append(route.copy())
            count += 1
        
        child1 = networkObj(route_network1, used_stops1, route_graph1, conn_type1, walk_type1)
        child2 = networkObj(route_network2, used_stops2, route_graph2, conn_type2, walk_type2)
        
        print("ALL ROUTES ARE DIFFERENT")
            
    # If there are some different routes, then split according to the number of different routes
    else:
        # PARENT 1
        # All not same routes from parent 1 will go to child 2
        for route in routes_not_same1:
            for connection in route:
                route_graph2.add_node(connection[0], **network1.graph.nodes[connection[0]])
                if connection[0] not in used_stops2:
                    used_stops2.append(connection[0])
                    
                route_graph2.add_node(connection[-1], **network1.graph.nodes[connection[-1]])
                if connection[-1] not in used_stops2:
                    used_stops2.append(connection[-1])
                
                route_graph2.add_edge(connection[0], connection[-1], **network1.graph.get_edge_data(connection[0], connection[-1]))
            route_network2.append(route.copy())
        
        # All same routes from parent 1 will go to child 1
        for route in routes_same1:
            for connection in route:
                route_graph1.add_node(connection[0], **network1.graph.nodes[connection[0]])
                if connection[0] not in used_stops1:
                    used_stops1.append(connection[0])
                    
                route_graph1.add_node(connection[-1], **network1.graph.nodes[connection[-1]])
                if connection[-1] not in used_stops1:
                    used_stops1.append(connection[-1])
                
                route_graph1.add_edge(connection[0], connection[-1], **network1.graph.get_edge_data(connection[0], connection[-1]))
            route_network1.append(route.copy())
            
        # PARENT 2
        # All not same routes from parent 2 will go to child 1
        for route in routes_not_same2:
            for connection in route:
                route_graph1.add_node(connection[0], **network2.graph.nodes[connection[0]])
                if connection[0] not in used_stops1:
                    used_stops1.append(connection[0])
                    
                route_graph1.add_node(connection[-1], **network2.graph.nodes[connection[-1]])
                if connection[-1] not in used_stops1:
                    used_stops1.append(connection[-1])
                
                route_graph1.add_edge(connection[0], connection[-1], **network2.graph.get_edge_data(connection[0], connection[-1]))
            route_network1.append(route.copy())
        
        # All same routes from parent 2 will go to child 2
        for route in routes_same2:
            for connection in route:
                route_graph2.add_node(connection[0], **network2.graph.nodes[connection[0]])
                if connection[0] not in used_stops2:
                    used_stops2.append(connection[0])
                    
                route_graph2.add_node(connection[-1], **network2.graph.nodes[connection[-1]])
                if connection[-1] not in used_stops2:
                    used_stops2.append(connection[-1])
                
                route_graph2.add_edge(connection[0], connection[-1], **network2.graph.get_edge_data(connection[0], connection[-1]))
            route_network2.append(route.copy())
    
        child1 = networkObj(route_network1, used_stops1, route_graph1, conn_type1, walk_type1)
        child2 = networkObj(route_network2, used_stops2, route_graph2, conn_type2, walk_type2)
        
        print("SOME ROUTES ARE DIFFERENT")
    
    print("* Checking Child 1 for errors:", flush=True)
    # TODO: DELETE FOR TESTING IF GRAPH IS CONSISTENT WITH ITS LIST OF ROUTES
    print("Checking for graph and route consistency...", flush=True)
    check_graph_with_route(child1)
    print("Checking for graph and list of stops consistency...", flush=True)
    check_graph_with_stops(child1)
    print("Checking if order of routes is correct...", flush=True)
    check_order_route(child1.routes)
    print("Checking for list of stops and route consistency...", flush=True)
    check_stops_routes(child1)
    print("Checking for duplicates in routes...")
    check_duplicate_routes(child1)
    print()
    
    print("* Checking Child 2 for errors:", flush=True)
    # TODO: DELETE FOR TESTING IF GRAPH IS CONSISTENT WITH ITS LIST OF ROUTES
    print("Checking for graph and route consistency...", flush=True)
    check_graph_with_route(child2)
    print("Checking for graph and list of stops consistency...", flush=True)
    check_graph_with_stops(child2)
    print("Checking if order of routes is correct...", flush=True)
    check_order_route(child2.routes)
    print("Checking for list of stops and route consistency...", flush=True)
    check_stops_routes(child2)
    print("Checking for duplicates in routes...")
    check_duplicate_routes(child2)

    return child1, child2

In [ ]:
# MUTATION FUNCTION

# Modify the stop connections of a random route in the network
# Randomly select a route and randomly select a stop in that route
# Then randomly select another stop that is a not too far from the selected stop based on threshold
# Swap connections with that stop
def mutate(network_to_mutate, graph_of_stops):
    global set_walk_distance, MAX_DISTANCE
    
    # This is to copy the original network to be compared later
    test_graph_net2 = network_to_mutate.graph.copy()
    test_routes_net2 = [copy.deepcopy(r) for r in network_to_mutate.routes]
    test_stops_net2 = network_to_mutate.stops.copy()
    test_stops_connType2 = network_to_mutate.conn_type
    test_stops_walkType2 = network_to_mutate.walk_distance
    copy_test_network = networkObj(test_routes_net2, test_stops_net2, test_graph_net2, test_stops_connType2, test_stops_walkType2)
    
    # Randomly select a route
    random_route = random.choice(network_to_mutate.routes)

    
    # TODO: This is a temporary solution, it chooses either of the connections within the route except for the first and last connection
    # Randomly select a stop in the route
    # random_node_connection = random.choice(random_route) # Choose a random connection in the route
    index_array = list(range(1, len(random_route)-1))
    connection_index = random.choice(index_array)
    random_node_connection = random_route[connection_index]
    
    connection_stop_index = random.choice([0, -1]) # Choose whether the origin or destination node
    random_stop = random_node_connection[connection_stop_index] # The random stop to be swapped
    
    print("PICKED NODE TO SWAP - ", random_stop)
    
    # Get the old total distance
    old_total_distance = 0
    for connection in random_route:
        edge_data = network_to_mutate.graph.get_edge_data(connection[0], connection[-1])
        
        if edge_data == None:
            print("-- ERROR MISSING EDGE INFORMATION: ", connection[0], " - ", connection[-1])
        else:
            old_total_distance += edge_data['distance']
    
    
    # This is to get the subset distance (Distance without the connection with chosen random stop)
    if connection_stop_index == 0: # If it is the origin node in that connection (A, B, C and B is the chosen. Get B-C and A-B)
        prev_connection = random_route[connection_index-1] # Get the previous connection
        prev_node = prev_connection[0] # Get the origin node for that connection
        distance1 = network_to_mutate.graph.get_edge_data(prev_node, random_stop)['distance'] # Get the distance of the previous connection
        distance2 = network_to_mutate.graph.get_edge_data(random_stop, random_node_connection[-1])['distance'] # Get the distance of the current connection
        distance_to_subtract = distance1 + distance2
        
        # Previous connection: node1 - random_node
        # Current connection: random_node - node2
        node1 = prev_node # Setting the partner nodes
        node2 = random_node_connection[-1]
        
    else: #If it is the dest node in that connection
        distance1 = network_to_mutate.graph.get_edge_data(random_node_connection[0], random_stop)['distance'] # Get the distance of the current connection
        next_connection = random_route[connection_index+1] # Get the next route
        next_node = next_connection[-1] # Get the destination node for the next connection
        distance2 = network_to_mutate.graph.get_edge_data(random_stop, next_node)['distance']
        distance_to_subtract = distance1 + distance2
        
        # Previous connection: node1 - random_node
        # Current connection: random_node - node2
        node2 = next_node
        node1 = random_node_connection[0]
        
    subset_distance = old_total_distance - distance_to_subtract
    # Get the route id
    random_route_id = network_to_mutate.graph.get_edge_data(random_node_connection[0], random_node_connection[-1])['route_id']
    
    # Will try searching for a random stop 50 times (arbitrary)
    for i in range(50):
        # Get a random stop
        new_random_stop, new_random_stop_data = random.choice(list(graph_of_stops.nodes(data=True)))
        
        
        # Check if the new random stop is not within walking distance with the other stop in the route
        # Walking distance between node1 in route and stop to be swapped with
        source = (new_random_stop_data['lat'], new_random_stop_data['lon'])
        point = (graph_of_stops.nodes[node1]['lat'], graph_of_stops.nodes[node1]['lon'])
        walking_distance1 = geodesic(source, point).meters
        
        # Walking distance between node2 in route and stop to be swapped with
        source = (new_random_stop_data['lat'], new_random_stop_data['lon'])
        point = (graph_of_stops.nodes[node2]['lat'], graph_of_stops.nodes[node2]['lon'])
        walking_distance2 = geodesic(source, point).meters
        
        # Check if it already has been used in the route
        isCandidate = True
        for connection in random_route:
            if new_random_stop == connection[0] or new_random_stop == connection[-1]:
                isCandidate = False
                print("NEW NODE WAS ALREADY USED", flush=True)
                continue
        
        # Check if the edge has already been used
        has_edge1 = network_to_mutate.graph.has_edge(node1, new_random_stop)
        has_edge2 = network_to_mutate.graph.has_edge(new_random_stop, node2)
        
        if has_edge1:
            print(f"** Already have edge 1")
        if has_edge2:
            print(f"** Already have edge 2")
            
        if i == 49:
            print("------REACHED END")
        
        # If it is not within walking distances, has not been used in the same route, has a path, has no existing edge
        if walking_distance1 >= set_walk_distance and walking_distance2 >= set_walk_distance and isCandidate and nx.has_path(CITY_GRAPH, node1, new_random_stop) and nx.has_path(CITY_GRAPH, new_random_stop, node2) and not has_edge1 and not has_edge2:
            # DISTANCE 1: Get the total distance from point A to point B
            shortest_route1 = nx.shortest_path(CITY_GRAPH, node1, new_random_stop)
            distance_travelled1 = 0
            for i in range(len(shortest_route1)-1):
                node_data = CITY_GRAPH.nodes[shortest_route1[i]]
                next_node_data = CITY_GRAPH.nodes[shortest_route1[i+1]]
                distance_travelled1 += haversine(node_data['y'], node_data['x'], next_node_data['y'], next_node_data['x'])
            
            # DISTANCE 2: Get the total distance from point A to point B
            shortest_route2 = nx.shortest_path(CITY_GRAPH, new_random_stop, node2)
            distance_travelled2 = 0
            for i in range(len(shortest_route2)-1):
                node_data = CITY_GRAPH.nodes[shortest_route2[i]]
                next_node_data = CITY_GRAPH.nodes[shortest_route2[i+1]]
                distance_travelled2 += haversine(node_data['y'], node_data['x'], next_node_data['y'], next_node_data['x'])
                
            
            # If its within the 15km distance, then this new stop can be used
            if subset_distance + distance_travelled1 + distance_travelled2 <= MAX_DISTANCE:
                # Add the new stop to the used stops
                if new_random_stop not in network_to_mutate.stops:
                    network_to_mutate.stops.append(new_random_stop)
                
                # Modify the graph by adding the new node
                network_to_mutate.graph.add_node(new_random_stop, **graph_of_stops.nodes[new_random_stop])
                print("MUTATION: ADDED NEW NODE TO GRAPH ", new_random_stop)
                
                # Connect the new stop to the edges
                name1 = f"{shortest_route1[0]}_{shortest_route1[-1]}"
                name2 = f"{shortest_route2[0]}_{shortest_route2[-1]}"
                connection_count1 = network_to_mutate.graph.get_edge_data(node1, random_stop)['edge_id']
                connection_count2 = network_to_mutate.graph.get_edge_data(random_stop, node2)['edge_id']
                network_to_mutate.graph.add_edge(shortest_route1[0], shortest_route1[-1], road_path=shortest_route1, edge_name=name1, edge_id = connection_count1, route_id = random_route_id, distance = distance_travelled1) # Add edge
                network_to_mutate.graph.add_edge(shortest_route2[0], shortest_route2[-1], road_path=shortest_route2, edge_name=name2, edge_id = connection_count2, route_id = random_route_id, distance = distance_travelled2) # Add edge 
                
                #TODO: DELETE PRINT FOR TESTING
                print("MUTATION: CONNECTED NEW STOP TO EDGES")  
                
                # Remove the old connections and node if it has no other connections
                network_to_mutate.graph.remove_edge(node1, random_stop)
                network_to_mutate.graph.remove_edge(random_stop, node2)
                print("MUTATION: REMOVED OLD CONNECTIONS TO NODE")
                if (network_to_mutate.graph.degree(random_stop) == 0):
                    network_to_mutate.graph.remove_node(random_stop)                
                    network_to_mutate.stops.remove(random_stop)
                    
                    print("MUTATION: REMOVED OLD NODE ", random_stop)
                else:
                    print("MUTATION: DIDNT REMOVED OLD NODE ", random_stop)
                
                # TODO: Delete FOR TESTING ONLY
                unique_nodes_G1 = set(copy_test_network.graph.nodes) - set(network_to_mutate.graph.nodes)
                unique_edges_G1 = set(copy_test_network.graph.edges) - set(network_to_mutate.graph.edges)

                unique_nodes_G2 = set(network_to_mutate.graph.nodes) - set(copy_test_network.graph.nodes)
                unique_edges_G2 = set(network_to_mutate.graph.edges) - set(copy_test_network.graph.edges)
                print("TESTING DIFFERENCES BETWEEN ORIGINAL AND OLD GRAPH")
                
                # Display unique nodes and edges for G1
                print("Unique nodes in original (not in new):")
                for node in unique_nodes_G1:
                    print(node)

                print("Unique edges in original (not in new):")
                for edge in unique_edges_G1:
                    print(edge)

                # Display unique nodes and edges for G2
                print("Unique nodes in new (not in original):")
                for node in unique_nodes_G2:
                    print(node)

                print("Unique edges in new (not in original):")
                for edge in unique_edges_G2:
                    print(edge)
                
                # Modify the route
                if connection_stop_index == 0: #If its the origin node
                    random_route[connection_index-1] = shortest_route1 # Change the previous connection
                    random_route[connection_index] = shortest_route2 # Change the current connection
                    
                    #TODO: DELETE FOR TESTING
                    random_route2 = copy_test_network.routes[network_to_mutate.routes.index(random_route)]
                    print("ORIGINAL: ", random_route2[connection_index-1][0], " - ", random_route2[connection_index-1][-1])
                    print("ORIGINAL: ", random_route2[connection_index][0], " - ", random_route2[connection_index][-1])
                    print("MODIFIED: ", random_route[connection_index-1][0], " - ", random_route[connection_index-1][-1])
                    print("MODIFIED: ", random_route[connection_index][0], " - ", random_route[connection_index][-1])
                    
                else: #else If its the dest node
                    random_route[connection_index] = shortest_route1 # Change the current connection
                    random_route[connection_index+1] = shortest_route2 # Change the next connection
                    
                    #TODO: DELETE FOR TESTING
                    random_route2 = copy_test_network.routes[network_to_mutate.routes.index(random_route)]
                    print("ORIGINAL: ", random_route2[connection_index][0], " - ", random_route2[connection_index][-1])
                    print("ORIGINAL: ", random_route2[connection_index+1][0], " - ", random_route2[connection_index+1][-1])
                    print("MODIFIED: ", random_route[connection_index][0], " - ", random_route[connection_index][-1])
                    print("MODIFIED: ", random_route[connection_index+1][0], " - ", random_route[connection_index+1][-1])
                    
                print("MUTATION: MODIFIED THE NETWORK OBJECT'S ROUTE")
                
                        
                # TODO: DELETE FOR TESTING IF GRAPH IS CONSISTENT WITH ITS LIST OF ROUTES
                print("Checking for graph and route consistency...")
                check_graph_with_route(network_to_mutate)
                print("Checking for graph and list of stops consistency...")
                check_graph_with_stops(network_to_mutate)
                print("Checking if order of routes is correct...")
                check_order_route(network_to_mutate.routes)
                print("Checking for list of stops and route consistency...")
                check_stops_routes(network_to_mutate)
                print("Checking for duplicates in routes...")
                check_duplicate_routes(network_to_mutate)
                
                print("MUTATION DONE", flush=True)
                
                
                # Break the loop once we swap
                print()
                break  

#### Fitness Function

In [ ]:
# Fitness function

def select_highest_scoring_mutation(candidate_road_snapped_networks, num_failure_removal,
                                    weight_random_failure, weight_targeted_failure, weight_radius_of_gyration):
    max_fitness_score = -np.inf
    max_candidate_route_snapped_network = None

    for n in candidate_road_snapped_networks:
        fitness_score = compute_fitness_score(n, num_failure_removal,
                                              weight_random_failure, weight_targeted_failure, weight_radius_of_gyration)
        if fitness_score > max_fitness_score:
            max_fitness_score = fitness_score
            max_candidate_route_snapped_network = n

    return max_candidate_route_snapped_network

def compute_fitness_score(road_snapped_network_graph, num_failure_removal,
                          weight_random_failure, weight_targeted_failure, weight_connectivity):

    random_failure_robustness = compute_random_failure_robustness(road_snapped_network_graph, num_failure_removal)
    weighted_random_failure_robustness = weight_random_failure * random_failure_robustness

    targeted_failure_robustness = compute_targeted_failure_robustness(road_snapped_network_graph, num_failure_removal)
    weighted_targeted_failure_robustness = weight_targeted_failure * targeted_failure_robustness

    connectivity_score = compute_connectivity(road_snapped_network_graph)
    weighted_connectivity = weight_connectivity * connectivity_score
    
    # print("Random Failure Score: ", weighted_random_failure_robustness)
    # print("Target Failure Score: ", weighted_targeted_failure_robustness)
    # print("Connectivity: ", weighted_connectivity)

    # Will use this return for now to utilize target and random failure nodes 
    return weighted_connectivity - weighted_random_failure_robustness - weighted_targeted_failure_robustness
    # return weighted_radius_of_gyration


### WRITTEN IN PSEUDOCODE
def compute_connectivity(network):
    # External connectivity - measure how connected is the jeepney route network with other modes of transpo
    
    # Get the ratio of transportation stops to total stops in the network
    transpo_stops = [node for node, node_data in network.nodes(data=True) if node_data['isTranspo'] == True]
    total_stops = len(network.nodes(data=True))
    transpo_stop_ratio = len(transpo_stops) / total_stops

    # Get the average degree of all transportation stops in the network
    if len(transpo_stops) > 0:
        avg_transpo_degree = sum(network.degree(stop) for stop in transpo_stops) / len(transpo_stops)
    else:
        avg_transpo_degree = 1

    # Find a way to normalize the two values and combine them 

    # Internal connectivity - measure how connected is each jeepney route to other jeepney routes
                    
    # This counts how many nodes have intersections (Meaning node is connected to more than one route by route ID)
    num_intersections = 0
    for node, node_data in network.nodes(data=True):
        connected_edges = network.edges(node)
        unique_route_id = []
        
        for edge in connected_edges:
            route_id = network[edge[0]][edge[1]]['route_id']
            
            if route_id not in unique_route_id:
                unique_route_id.append(route_id)
                
        if len(unique_route_id) > 1:
            num_intersections += 1

    
    # Change these weights based on what the expected values for 
    # the transpo_stop_ratio, avg_transpo_degree, and num_intersections will be
    external_weight = 0.5
    internal_weight = 0.5
    
    # TODO: Delete this
    # print("Transpo stop ratio: ", transpo_stop_ratio)
    # print("Num intersections: ", num_intersections)
    # print("Average degree: ", avg_transpo_degree)

    # Formula subject to change
    return external_weight * (transpo_stop_ratio * avg_transpo_degree) + internal_weight * num_intersections


def compute_random_failure_robustness(road_snapped_network_graph, num_removals):
    graph_copy = road_snapped_network_graph.copy() # Make a copy
    
    for i in range(num_removals):
        selected_node = random.choice(list(graph_copy.nodes()))
        graph_copy.remove_node(selected_node)

    diameter, avg_path_length = compute_network_statistics(graph_copy)
    return compute_failure_robustness(graph_copy, diameter)

def compute_targeted_failure_robustness(road_snapped_network_graph, num_removals):
    graph_copy = road_snapped_network_graph.copy() # Make a copy
    
    for i in range(num_removals):
        node_degrees = graph_copy.degree()
        # Iterate over the DegreeView object to find the maximum degree
        max_degree = max(degree for _, degree in node_degrees)
        max_degree_node = get_node_with_degree(node_degrees, max_degree)
        graph_copy.remove_node(max_degree_node)

    diameter, avg_path_length = compute_network_statistics(graph_copy)
    return compute_failure_robustness(graph_copy, diameter)

def compute_failure_robustness(road_snapped_network_graph, max_path_length):
    return float(max_path_length) / float(len(road_snapped_network_graph) - 1)

def compute_network_statistics(road_snapped_network_graph):
    path_lengths = get_path_lengths(road_snapped_network_graph) # Get the sum of all possible
    avg_path_length = np.mean(path_lengths)
    max_path_length = max(path_lengths)

    #network_size = len(path_lengths)
    #gcc = sorted(nx.connected_component_subgraphs(road_snapped_network_graph), key=len, reverse=True)
    #giant_component_fraction = float(float(gcc[0].order()) / float(network_size))
    #return max_path_length, avg_path_length, giant_component_fraction
    return max_path_length, avg_path_length

def get_node_with_degree(node_degrees, degree):
    # Iterate over the DegreeView object to find the node with the specified degree
    for node, _ in node_degrees:
        if _ == degree:
            return node
    return None  # Return None if no node with the specified degree is found

def get_path_lengths(snapped_road_network_graph):
    return [sum(nx.single_source_shortest_path_length(snapped_road_network_graph, n).values())
            for n in snapped_road_network_graph]

### Graph Error Checks

In [ ]:
# ERROR CHECK - Checking if there are duplicate routes in the list of routes
def check_duplicate_routes(network):
    seen = set()
    duplicates = []
    error = "xxx"

    for route in network.routes:
        for lst in route:
            tpl = tuple(lst)
            if tpl in seen:
                duplicates.append(lst)
            else:
                seen.add(tpl)

    if duplicates:
        print(f"---- ERROR{error} Duplicate lists found")
        for duplicate in duplicates:
            print(duplicate)

In [ ]:
# ERROR Check - Checks if routes is consistent with its stops
def check_stops_routes(network):
    error = "xxx"
    # Checks if each node in the route is in the list of stops
    for route in network.routes:
        for connection in route:
            if connection[0] not in network.stops:
                print(f"---- ERROR{error} MISSING ROUTE ORIGIN STOP IN LIST OF STOPS ", connection[0])
            if connection[-1] not in network.stops:
                print("-- MISSING ROUTE DEST STOP IN LIST OF STOPS ", connection[-1])
    
    

In [ ]:
# ERROR Check - Checks if the routes are in correct order
def check_order_route(routes):
    error = "xxx"
    for route in routes:
        for connection in route:
            if route.index(connection) > 0:
                if connection[0] != prev_connection[-1]:
                    print(f"---- ERROR{error} WRONG ORDER DETECTED --")
                    print(prev_connection[0], " - ", prev_connection[-1])
                    print(connection[0], " - ", connection[-1])
                    print("--------------------------")
            prev_connection = connection
        

In [ ]:
# ERROR CHECK - Checks the consistency of the network with its routes
def check_graph_with_route(_network):
    error = "xxx"
    for route in _network.routes:
        for connection in route:
            if not _network.graph.has_node(connection[0]):
                print(f"---- ERROR{error} MISSING NODE IN GRAPH: ", connection[0])
            if not _network.graph.has_node(connection[-1]):
                print(f"---- ERROR{error} MISSING NODE IN GRAPH: ", connection[-1])
            if not _network.graph.has_edge(connection[0], connection[-1]):
                print(f"---- ERROR{error} MISSING EDGE IN GRAPH: ", connection[0], " - ", connection[-1])
                
            if _network.graph.get_edge_data(connection[0], connection[-1]) == None:
                print(f"---- ERROR{error} MISSING EDGE INFORMATION: ", connection[0], " - ", connection[-1])

In [ ]:
# ERROR CHECK - Checks the consistency of the network with its list of stops
def check_graph_with_stops(_network):
    error = "xxx"
    # Checks if all stops in the list is in the graph
    for stop in _network.stops:
        if not _network.graph.has_node(stop):
            print(f"---- ERROR{error} MISSING LIST STOP IN GRAPH: ", stop)
            
    # Checks if all nodes in the graph are in the list
    for node, node_data in _network.graph.nodes(data=True):
        if node not in _network.stops:
            print(f"---- ERROR{error} MISSING GRAPH NODE IN LIST: ", node)

### Simplicity

In [ ]:
# TODO: WORKING IN PROGRESS
def simplicity_metric(network):
    routes = network.routes
    
    for route in routes:
        for i in range(len(route) - 1):
            u, v = route[i], route[i + 1]
            
            if CITY_GRAPH.has_edge(u, v):
                edge_data = CITY_GRAPH.get_edge_data(u, v)
            else:
                # Skip if there's no direct edge between u and v
                continue
            
            # Edge data might have multiple edges with different keys
            for key in edge_data:
                road_name = edge_data[key].get('name', 'Unnamed Road')
            
            
            
            # Compare

### Network Analysis Metrics

In [ ]:
# Longest route, shortest route, average route length, network diamater

### GA and Fitness Function Tests

In [ ]:

# #Weights and Fitness Function configuration
# num_failure_removal = 4
# weight_random_failure = 0.15
# weight_targeted_failure = 0.15
# weight_connectivity = 0.7

# i = 0
# for network in list_of_networks_Manila:
#     print(f"NETWORK {i}")
#     road_snapped_network_graph = network.graph
#     score = compute_fitness_score(road_snapped_network_graph, num_failure_removal, weight_random_failure, weight_targeted_failure, weight_connectivity)
#     print(f"Network {i} score: {score}")
#     print()
#     i += 1

In [ ]:
# #TODO: DELETE FOR GA TESTING ONLY
# #Testing crossover and mutate

# network1= list_of_networks_Manila[0]
# network2= list_of_networks_Manila[1]

# test_graph_net1 = network1.graph.copy()
# test_routes_net1 = [copy.deepcopy(r) for r in network1.routes]
# test_stops_net1 = network1.stops.copy()
# copy_test_network1 = networkObj(test_routes_net1, test_stops_net1, test_graph_net1, list_of_networks_Manila[0].conn_type)

# test_graph_net2 = network2.graph.copy()
# test_routes_net2 = [copy.deepcopy(r) for r in network2.routes]
# test_stops_net2 = network2.stops.copy()
# copy_test_network2 = networkObj(test_routes_net2, test_stops_net2, test_graph_net2, list_of_networks_Manila[1].conn_type)

# child1, child2 = crossover_split_index(copy_test_network1,copy_test_network2)

# map_center = (14.599512, 120.984222)

# print("--------------- START")
# if len(child1.graph) != len(child1.stops):
#         print("-----CHILD 1 STOPS AND GRAPH NOT EQUAL")
# if len(child2.graph) != len(child2.stops):
#     print("-----CHILD 2 STOPS AND GRAPH NOT EQUAL")
# print("--------------- END")
# print()
        
# # -----Child 1 Display -------------
# m = folium.Map(location=map_center, zoom_start=1, tiles='openstreetmap')
# add_markers(child1.stops, child1.graph)
    
# for route in child1.routes:
#     for connection in route:
#         ox.plot_route_folium(CITY_GRAPH, connection, route_map=m, tiles='openstreetmap', route_color="green")

# m.save(f"GA TEST (DELETE LATER)/child1_crossover.html")

# # ------Child 2 Display ------------
# m = folium.Map(location=map_center, zoom_start=1, tiles='openstreetmap')
# add_markers(child2.stops, child2.graph)
    
# for route in child2.routes:
#     for connection in route:
#         ox.plot_route_folium(CITY_GRAPH, connection, route_map=m, tiles='openstreetmap', route_color="green")

# m.save(f"GA TEST (DELETE LATER)/child2_crossover.html")


# # # ---------Graph test---------------
# # # TODO: Delete FOR TESTING ONLY
# # unique_nodes_G1 = set(child1.graph.nodes) - set(child2.graph.nodes)
# # unique_edges_G1 = set(child1.graph.edges) - set(child2.graph.edges)

# # unique_nodes_G2 = set(child2.graph.nodes) - set(child1.graph.nodes)
# # unique_edges_G2 = set(child2.graph.edges) - set(child1.graph.edges)
# # print("TESTING DIFFERENCES BETWEEN Child1 AND Child2")

# # # Display unique nodes and edges for G1
# # print("Unique nodes in child1 (not in child2):")
# # for node in unique_nodes_G1:
# #     print(node)

# # print("Unique edges in child1 (not in child2):")
# # for edge in unique_edges_G1:
# #     print(edge)

# # # Display unique nodes and edges for G2
# # print("Unique nodes in child2 (not in child1):")
# # for node in unique_nodes_G2:
# #     print(node)

# # print("Unique edges in child2 (not in child1):")
# # for edge in unique_edges_G2:
# #     print(edge)

# # --------Mutate test --------------
# for i in range(1):
#     print("Child 1 ATTEMPT MUTATION")
#     mutate(child1, graph_of_stops_Manila)
#     print("-------------------------")
#     print()
    
#     print("Child 2 ATTEMPT MUTATION")
#     mutate(child2, graph_of_stops_Manila)
#     print("-------------------------")
#     print()
    
#     if len(child1.graph) != len(child1.stops):
#         print("-----CHILD 1 STOPS AND GRAPH NOT EQUAL")
#     if len(child2.graph) != len(child2.stops):
#         print("-----CHILD 2 STOPS AND GRAPH NOT EQUAL")

#     # # -----Child 1 Mutation Display -------------
#     # m = folium.Map(location=map_center, zoom_start=1, tiles='openstreetmap')
#     # add_markers(child1.stops, child1.graph)
        
#     # for route in child1.routes:
#     #     for connection in route:
#     #         ox.plot_route_folium(CITY_GRAPH, connection, route_map=m, tiles='openstreetmap', route_color="green")

#     # m.save(f"GA TEST (DELETE LATER)/child1_mutation{i}.html")

#     # # ------Child 2 Mutation Display ------------
#     # m = folium.Map(location=map_center, zoom_start=1, tiles='openstreetmap')
#     # add_markers(child2.stops, child2.graph)
        
#     # for route in child2.routes:
#     #     for connection in route:
#     #         ox.plot_route_folium(CITY_GRAPH, connection, route_map=m, tiles='openstreetmap', route_color="green")

#     # m.save(f"GA TEST (DELETE LATER)/child2_mutation{i}.html")



### Visualizations

In [ ]:
# Function to plot/visualize connected zones on the map
import random

# This is to better visualize the networks
def plot_connected_zones_network_on_map(graph, initial_location=[0, 0], zoom_start=10):
    # Create a map centered at the initial location
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=zoom_start, tiles='openstreetmap')
    
    #Colours for Visualization
    colors = [
    "Red", "Green", "Blue", "Yellow", "Orange", "Purple", "Cyan", "Magenta", "Maroon",
    "Olive", "Lime", "Teal", "Navy", "Aqua", "Fuchsia", "Coral", "Indigo", "Violet"]
    
    color_map = {}

    # Iterate over the nodes in the network
    for node, data in graph.nodes(data=True):
        # Check if the node has a geometry attribute
        if 'geometry' in data:
            # Get the geometry of the node
            geometry = data['geometry']

            # Check the geometry type and plot accordingly
            if geometry.geom_type == 'Point':
                # Plot a marker for points    
                #folium.Marker(location=[geometry.y, geometry.x], popup=f"{data['name']}").add_to(m)
                continue
            elif geometry.geom_type in ['Polygon', 'MultiPolygon']:
                
                network_id = data["network_id"]
                
                if network_id not in color_map:
                    color = random.choice(colors)
                    color_map[network_id] = color
                else:
                    color = color_map[network_id]
                
                if geometry.geom_type == 'Polygon':
                    polygons = [geometry]
                else:
                    polygons = geometry.geoms

                for polygon in polygons:
                    coordinates = []
                    for point in polygon.exterior.coords:
                        coordinates.append([point[1], point[0]])
                    folium.Polygon(locations=coordinates, fill=True, color=color, fill_opacity=0.4).add_to(m)

    # Return the map
    return m


In [ ]:
# This is to visualize the stops
def plot_stops_on_map(network_map, stops, initial_location=[0, 0], zoom_start=10):
    # Iterate over the nodes in the network
    for stop in stops:
        folium.Marker(location=[stop.lat, stop.long], popup=f"{stop.isTranspo}").add_to(network_map)
        
    return network_map

## Main

### Manila Test

#### Reading Data

In [ ]:
# File Paths
Manila_pikl_filepath = "Saved Networks/Manila/"
Manila_map_filepath = "Saved Maps/Manila/"

In [ ]:
# READING DATA (All Amenities in Manila)
#CHANGE THIS BACK
merged_amenities_points_gdf = gpd.read_file('./City Data/Manila City/Manila_point.geojson')
merged_amenities_polygons_gdf= gpd.read_file('././City Data/Manila City/Manila_polygon.geojson')

merged_amenities_polygons_gdf['amenity_points'] = None

In [ ]:
# Getting total area to be used for the area connection type
merged_amenities_polygons_gdf['area'] = degrees_to_meters(merged_amenities_polygons_gdf['geometry'].area)
manila_area_sum = merged_amenities_polygons_gdf['area'].sum()

manila_area_sum

In [ ]:
# Create spatial index for points
idx = rtree_index.Index()
for j, point in merged_amenities_points_gdf.iterrows():
    idx.insert(j, point['geometry'].bounds)

# Iterate over polygons
for i, polygon in merged_amenities_polygons_gdf.iterrows():
    points_within_polygon = []
    
    # Iterate over points within the bounding box of the polygon
    for j in idx.intersection(polygon['geometry'].bounds):
        point = merged_amenities_points_gdf.loc[j]
        if polygon['geometry'].intersects(point['geometry']):
            points_within_polygon.append(j)
    
    merged_amenities_polygons_gdf.at[i, 'amenity_points'] = points_within_polygon

#### Importing Manila Zone Network Data

In [ ]:
# ------- LEVEL 1 
# IMPORT INITIAL NETWORK
initial_network_Manila = import_networks(f"{Manila_pikl_filepath}Manila_Initial_Network.pkl")

# IMPORT FILTERED NETWORK
filtered_manila_amenities_network = import_networks(f"{Manila_pikl_filepath}Manila_Filtered_Network.pkl")

# IMPORT COMBINED AMENITIES NETWORK
combined_graph_Manila = import_networks(f"{Manila_pikl_filepath}Manila_Combined_Amenities_Network.pkl")

# ------- LEVEL 2
# IMPORT POPULATION GRAPH
pop_graph = import_networks(f"{Manila_pikl_filepath}Manila_Population_Graph.pkl")

# IMPORT ZONE NETWORK
graph_networks_of_polygons_Manila = import_networks(f"{Manila_pikl_filepath}Manila_Zone_Network.pkl")
networks_map_Manila = plot_connected_zones_network_on_map(graph_networks_of_polygons_Manila, initial_location=[0, 0], zoom_start=100)

In [ ]:
# Importing Stop list and Graph (Only to test the same stops)
list_of_stops_Manila = import_networks(f"{Manila_pikl_filepath}stop_list_Manila.pkl")
graph_of_stops_Manila = import_networks(f"{Manila_pikl_filepath}stop_graph_Manila.pkl")

# Visualize the stops
stops_map = plot_stops_on_map(networks_map_Manila, list_of_stops_Manila, initial_location=[0, 0], zoom_start=100)
stops_map.save(f"{Manila_map_filepath}stops_map.html") # Save the map to an HTML file

In [ ]:
# Importing Generated Routes (Only to test the same routes)
# WALKING_DISTANCES -> 300,550,800
# CONNECTION_TYPES -> "Default", "Area", "Degree", "Mixed"
import_conn_type = CONNECTION_TYPES[0]
import_walk_distance = WALKING_DISTANCES[0]

if import_conn_type == 'Mixed':
    list_of_networks_Manila = import_networks(f"{Manila_pikl_filepath}Manila_Route_networks_{import_conn_type}.pkl")
else:
    list_of_networks_Manila = import_networks(f"{Manila_pikl_filepath}Manila_Route_networks_{import_conn_type}_{import_walk_distance}.pkl")

In [ ]:
# Importing GA Results (Only to analyze shared results)
# WALKING_DISTANCES -> 300,550,800
# CONNECTION_TYPES -> "Default", "Area", "Degree", "Mixed"
import_conn_type = CONNECTION_TYPES[0]
import_walk_distance = WALKING_DISTANCES[0]

if import_conn_type == 'Mixed':
    population = import_networks(f"{Manila_pikl_filepath}Manila_GA_Route_networks_{import_conn_type}.pkl")
else:
    population = import_networks(f"{Manila_pikl_filepath}Manila_GA_Route_networks_{import_conn_type}_{import_walk_distance}.pkl")


#### Stop Placement (Only to generate new stops)

In [ ]:
# LEVEL 3 - CREATING STOPS TO BE PLACED ON ZONES
graph_of_stops_Manila = nx.Graph()
add_points_to_graph(initial_network_Manila, graph_networks_of_polygons_Manila) # Add first all transportation stops
list_of_stops_Manila = []
place_stops_on_roads(graph_networks_of_polygons_Manila, graph_of_stops_Manila, list_of_stops_Manila) # Adds stops graph_of_stops

# Visualize the stops
stops_map = plot_stops_on_map(networks_map_Manila, list_of_stops_Manila, initial_location=[0, 0], zoom_start=100)
stops_map.save(f"{Manila_map_filepath}stops_map.html") # Save the map to an HTML file

In [ ]:
#Export stops to pickle
file_path = f'{Manila_pikl_filepath}stop_list_Manila.pkl'
with open(file_path, 'wb') as f:
    pickle.dump(list_of_stops_Manila, f)
    
file_path = f'{Manila_pikl_filepath}stop_graph_Manila.pkl'
with open(file_path, 'wb') as f:
    pickle.dump(graph_of_stops_Manila, f)

In [ ]:
len(list_of_stops_Manila)

In [ ]:
len(graph_of_stops_Manila)

#### Stop Connection (Only to generate new routes)

In [ ]:
# LEVEL 4 - CONNECTING STOPS INTO A NETWORK

# Configuration
set_walk_distance = WALKING_DISTANCES[1]
num_of_networks = 100
conn_type = CONNECTION_TYPES[3]
max_stops = 20
max_routes = 30 # temporary, should be 30
map_html_location = f"Generated Route Networks HTML/Manila/{conn_type}/"
        
# Generate route network
list_of_networks_Manila = []

print(f"CHOSEN CONNECTION TYPE: {conn_type}")
current_network_count = 0
for _ in range(num_of_networks):
    route_count = 0 # Route is the connection between list of stops
    connection_count = 0 # Connection is the connection between two stops / nodes
    
    print(f"NETWORK {current_network_count}")
    if conn_type == "Mixed":
        temp_conn_type = random.choice(CONNECTION_TYPES[:-1])
        temp_walk_type = random.choice(WALKING_DISTANCES)
        print(f"NETWORK CONNECTION TYPE: {temp_conn_type}")
        route_network, route_graph = generate_route_network(list_of_stops_Manila, set_walk_distance, max_stops, max_routes, graph_of_stops_Manila, manila_area_sum, temp_conn_type) # Default max walking distance is 300m
        used_stops = add_stops_to_list(route_network)
        new_network = networkObj(route_network, used_stops, route_graph, temp_conn_type, temp_walk_type)
    else:
        route_network, route_graph = generate_route_network(list_of_stops_Manila, set_walk_distance, max_stops, max_routes, graph_of_stops_Manila, manila_area_sum, conn_type) # Default max walking distance is 300m
        used_stops = add_stops_to_list(route_network)
        new_network = networkObj(route_network, used_stops, route_graph, conn_type, set_walk_distance)
    
    # ERROR CHECKS----------
    #print("Performing error checks...")
    #check_graph_with_route(new_network)
    #check_graph_with_stops(new_network)
    #check_order_route(new_network.routes)
    
    print()
    list_of_networks_Manila.append(new_network) # Append to list of networks
    current_network_count += 1

#Export networks and graphs using pickl
if conn_type == 'Mixed':
    export_networks(list_of_networks_Manila, f"{Manila_pikl_filepath}Manila_Route_networks_{conn_type}.pkl")
else:
    export_networks(list_of_networks_Manila, f"{Manila_pikl_filepath}Manila_Route_networks_{conn_type}_{set_walk_distance}.pkl")

In [ ]:
# Creating Maps for visualization

i = 1
for route_network in list_of_networks_Manila:
    map_center = (14.599512, 120.984222)
    m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

    # Plotting in the Map
    add_markers(route_network.stops, route_network.graph)
        
    for route in route_network.routes:
        for connection in route:
            ox.plot_route_folium(CITY_GRAPH, connection, route_map=m, tiles='openstreetmap', route_color="green")

    m.save(f"{map_html_location}Route Map-{i}.html")
    i += 1

#### Genetic Algorithm (Only to generate new GA results)

In [ ]:
# TESTING GA ---------------------------------------------------------]
# There should a pickle file already of the latest networks

# GA Configuration
population_size = 100
num_elites = 2
num_generations = 5
mutation_probability = 0.2
num_mutations_probabilities = [0.05, 0.15, 0.4, 0.2, 0.2]
num_crossovers_probabilities = [0.1, 0.1, 0.4, 0.2, 0.2]
mutation_threshold_dist = 300
with_elitism = False
with_growing_population = False
num_mutations_per_generation = 5

#Weights and Fitness Function configuration
num_failure_removal = 4
weight_random_failure = 0.15
weight_targeted_failure = 0.15
weight_connectivity = 0.7

population = perform_genetic_algorithm(list_of_networks_Manila, graph_of_stops_Manila, population_size, num_elites, num_generations, mutation_probability, 
                              num_mutations_probabilities, num_crossovers_probabilities, mutation_threshold_dist, num_failure_removal,
                                      weight_random_failure, weight_targeted_failure, weight_connectivity, 40,
                              with_elitism, with_growing_population, num_mutations_per_generation)

In [ ]:
# Exporting GA Results
unique_types = {obj.conn_type for obj in list_of_networks_Manila}
if len(unique_types) > 1:
    export_conn_type = "Mixed"
    
    export_networks(population, f"{Manila_pikl_filepath}Manila_GA_Route_networks_{export_conn_type}.pkl")
    print(f"Exported: Manila_GA_Route_networks_{export_conn_type}.pkl")
else:
    export_conn_type = list_of_networks_Manila[0].conn_type
    export_walk_distance = list_of_networks_Manila[0].walking_distance

    export_networks(population, f"{Manila_pikl_filepath}Manila_GA_Route_networks_{export_conn_type}_{export_walk_distance}.pkl")
    print(f"Exported: Manila_GA_Route_networks_{export_conn_type}_{export_walk_distance}.pkl")

#### Visualization of Genetic Algorithm Results

In [ ]:
# Visualization
map_html_location = "GA Result Route Networks HTML/Manila/"

i = 1
for network in population:
    map_center = (14.599512, 120.984222)
    m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')
    add_markers(network.stops, network.graph)
    
    for route in network.routes:
        for connection in route:
            ox.plot_route_folium(CITY_GRAPH, connection, route_map=m, tiles='openstreetmap', route_color="green")

    m.save(f"{map_html_location}GA Map-{i}.html")
    i += 1